mnist cnn

In [2]:
import torch
import torchvision

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = torchvision.datasets.MNIST(
    root = 'MNIST_data',
    train = True, 
    transform = torchvision.transforms.ToTensor(), 
    download = True
)
mnist_test = torchvision.datasets.MNIST(
    root = 'MNIST_data', 
    train = False, 
    transform = torchvision.transforms.ToTensor(), 
    download = True
)

In [5]:
data_loader = torch.utils.data.DataLoader(
    dataset = mnist_train, 
    batch_size = batch_size, 
    shuffle = True, 
    drop_last = True
)

In [6]:
class CNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [7]:
model = CNN().to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print("Epoch: {:2d} cost = {}".format(epoch + 1, avg_cost))
print("Learning Finished")

Epoch:  1 cost = 0.22563090920448303
Epoch:  2 cost = 0.06301800906658173
Epoch:  3 cost = 0.04623953998088837
Epoch:  4 cost = 0.037524543702602386
Epoch:  5 cost = 0.03135891631245613
Epoch:  6 cost = 0.026088563725352287
Epoch:  7 cost = 0.021748894825577736
Epoch:  8 cost = 0.01845337264239788
Epoch:  9 cost = 0.01605328731238842
Epoch: 10 cost = 0.013379793614149094
Epoch: 11 cost = 0.010189618915319443
Epoch: 12 cost = 0.009682618081569672
Epoch: 13 cost = 0.00914840865880251
Epoch: 14 cost = 0.00654180720448494
Epoch: 15 cost = 0.005453532561659813
Learning Finished


In [10]:
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim = 1) == Y_test
    accuracy = correct_prediction.float().mean()

    print("accuracy : {:.4f}%".format(accuracy.item() * 100))


accuracy : 98.6900%


In [11]:
del X_test
del Y_test
torch.cuda.empty_cache()
# 모델이 너무 깊은 경우에는 accuracy가 오히려 떨어질 수도 있음, 효율적인 구조롤 잘 쌓아야 함

print(mnist_test.data.view(-1, 1, 28, 28)[0])

tensor([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,

In [12]:
with torch.no_grad():
    X_test = (mnist_test.data.view(-1, 1, 28, 28).float()/255).to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim = 1) == Y_test
    accuracy = correct_prediction.float().mean()

    print("accuracy : {:.4f}%".format(accuracy.item() * 100))


accuracy : 99.0300%
